<a href="https://colab.research.google.com/github/201524495/201524495/blob/main/text_to_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text-to-Video Hands‑On

이 노트북은 텍스트 프롬프트로 짧은 비디오를 생성해보며, 확산 모델의 핵심 개념을 직접 실습할 수 있게 구성했습니다.

구성:
1. 빠른 시연 (데모) — 바로 결과 만들어 보기
2. 개념 이해 — 확산(diffusion), 스케줄러, CFG, 시드, 프레임 수 등
3. **TODO 실습** — 작은 함수/로직을 직접 작성하여 이해도 높이기

## 0) 런타임 설정 및 GPU 확인
- "런타임" → "런타임 유형 변경" → 하드웨어 가속기: **GPU** (T4)
- 아래 셀을 실행하여 GPU 인식 및 기본 환경을 확인합니다.

In [ ]:
import torch, platform, sys, subprocess, os, random
print(f"Python        : {sys.version.split()[0]}")
print(f"PyTorch avail?: {torch.cuda.is_available()}")
print(f"GPU name      : {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
print(f"CUDA version  : {torch.version.cuda}")
print(f"Platform      : {platform.platform()}")
if torch.cuda.is_available():
    !nvidia-smi -L
    !nvidia-smi

## 1) 의존성 설치
Colab 기본 PyTorch를 그대로 쓰고, 필요한 라이브러리만 설치합니다.

**설치 패키지**
- `diffusers` : 모델 로딩/추론 파이프라인
- `transformers`, `accelerate`, `safetensors`
- `imageio`, `imageio-ffmpeg` : 비디오 저장
- `decord` : 프레임 미리보기

In [ ]:
!pip -q install -U diffusers transformers accelerate safetensors imageio imageio-ffmpeg decord

## 2) Demo
`diffusers`의 **ModelScope Text2Video** 파이프라인을 사용합니다.

> 만약 VRAM이 부족하면 `num_frames`, `height`, `width`, `num_inference_steps`를 줄이세요.

In [ ]:
from diffusers import DiffusionPipeline
import torch, os
from diffusers.utils import export_to_video

model_id = "damo-vilab/text-to-video-ms-1.7b"
pipe = DiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    variant="fp16",
)

pipe.enable_attention_slicing()
pipe.enable_vae_slicing()
pipe.enable_model_cpu_offload()

prompt = "a cute corgi surfing on a tropical wave, cinematic lighting, 4k"
negative_prompt = "low quality, text, watermark"
seed = 42
generator = torch.Generator(device="cuda").manual_seed(seed)

result = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=25,
    num_frames=16,
    guidance_scale=9.0,
    height=320, width=512,
    generator=generator,
)
frames = result.frames[0]
video_path = "demo_ms_t2v.mp4"
export_to_video(frames, video_path, fps=8)
print(f"Saved: {os.path.abspath(video_path)}")

### Demo
Colab에서 mp4를 직접 재생합니다.

In [ ]:
from IPython.display import HTML
from base64 import b64encode
def show_video(path):
    mp4 = open(path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML(f"""
    <video width=640 controls><source src='{data_url}' type='video/mp4'></video>
    """)

show_video("demo_ms_t2v.mp4")

## 3) 직접 만들기 (TODO 실습)
아래 함수/코드를 **TODO**를 채워가며 완성해 보세요. 각 함수는 비교적 짧고, 주석에 힌트가 있습니다.

완성 후에는 아래 "테스트" 셀로 바로 확인할 수 있습니다.

In [ ]:
import imageio
from typing import List

def make_generator(seed: int, device: str = "cuda"):
    """TODO: 주어진 seed로 torch.Generator를 생성해 반환하세요."""
    ### YOUR CODE HERE ###
    # hint: torch.Generator(device).manual_seed(seed)
    return generator

def export_to_mp4(frames: List, path: str, fps: int = 8):
    """TODO: imageio.get_writer를 사용해 frames를 mp4로 저장하세요.
    frames = [frame1, frame2, ... ]
    """
    with imageio.get_writer(path, fps=fps, codec="libx264", macro_block_size=None) as writer:
        ### YOUR CODE HERE ###
        # array = each frame in frames
        # writer.append_data(array)
        pass

def apply_style(prompt: str, style: str = "cinematic") -> str:
    """TODO: style에 따라 prompt를 가볍게 보강해 반환하세요."""

    style_map = {
        "cinematic": ", cinematic lighting, 4k, depth of field, high contrast",
        "animation": ", 2d animation, vibrant colors, clean lines",
        "studio": ", studio lighting, soft shadows, color graded",
        "film": ", 35mm film grain, natural lighting, shallow depth of field",
        "sketch": ", pencil sketch, monochrome, line art style",
    }

    styled_prompt = None # prompt + additional_style_prompt

    ### YOUR CODE HERE ###
    return styled_prompt

def sweep(values, fn):
    return [fn(v) for v in values]

In [ ]:
from diffusers import DiffusionPipeline
import torch

def load_ms_t2v(dtype=torch.float16, cpu_offload=True):
    """ModelScope Text2Video 파이프라인을 로드합니다.
    TODO: dtype/메모리절약 옵션을 인자로 반영하고 return 하세요.
    """
    model_id = "damo-vilab/text-to-video-ms-1.7b"

    kwargs = {"torch_dtype": dtype}
    if dtype == torch.float16:
        kwargs["variant"] = "fp16"

    ### YOUR CODE HERE ###
    # pipe = DiffusionPipeline.from_pretrained(...)

    pipe.enable_attention_slicing()
    pipe.enable_vae_slicing()
    if cpu_offload:
        pipe.enable_model_cpu_offload()
    else:
        if torch.cuda.is_available():
            pipe.to("cuda")

    return pipe
\
def generate_video(pipe, prompt: str, negative_prompt: str = None,
                   num_frames: int = 16, steps: int = 25, guidance_scale: float = 9.0,
                   height: int = 320, width: int = 512, seed: int = 0, fps: int = 8,
                   save_path: str = "output.mp4"):
    """파이프라인으로 비디오를 생성하고 파일로 저장한 뒤 경로를 반환합니다.
    TODO: make_generator, pipe 호출, export_to_mp4를 사용하세요.
    """

    device = "cuda" if torch.cuda.is_available() else "cpu"

    ### YOUR CODE HERE ###
    # gen = ...
    # result = pipe(prompt=..., negative_prompt=..., num_inference_steps=..., num_frames=..., guidance_scale=..., height=..., width=..., generator=gen)
    # frames = result.frames[0]
    # export_to_mp4(frames, save_path, fps=fps)

    return save_path

In [ ]:
pipe = load_ms_t2v()
p = apply_style("a sleepy cat playing piano", style="cinematic")
out_path = generate_video(pipe, p, negative_prompt="low quality, watermark", seed=123, fps=8)
show_video(out_path)

## 4) 직접 해보기
다음 항목들을 수정해가며 **무엇이 어떻게 바뀌는지** 관찰해 보세요.

- `guidance_scale`: 5 ~ 13 사이를 돌려보세요. 너무 크면 부자연스러울 수 있어요.
- `num_inference_steps`: 생성 step의 수. 15, 25, 35를 비교해보세요.
- `num_frames`: 생성할 프레임의 수.
- `height/width`: 256×256, 320×512 등 해상도 변화.
- 프롬프트 엔지니어링: 장면, 스타일(수채화, 스톱모션, 저녁 역광 등) 바꿔보기.
- **부정 프롬프트**(negative prompt): 텍스트/워터마크/낮은 품질 제거에 도움.


In [ ]:
pipe = load_ms_t2v()
p = apply_style("a sleepy cat playing piano", style="cinematic")
demo_path = generate_video(...) # your arguments here
show_video(demo_path)

## 5) 문제 해결 (Troubleshooting)
- **메모리 부족(OOM)**: 프레임/해상도/스텝/가이던스 스케일을 줄이세요. `enable_model_cpu_offload()` 유지.
- **느림**: `enable_model_cpu_offload()`는 느려질 수 있습니다. 여유가 되면 끄고 실행해보세요.
- **결과가 마음에 들지 않음**: 프롬프트를 구체화하고, 부정 프롬프트로 원치 않는 요소를 제거하세요.
- **재현 안 됨**: 시드를 고정하세요.
